# Otsitavate lemmade ja alamlemmade nimekirja moodustamine 

[![License: BSD-2-Clause](https://img.shields.io/badge/License-BSD--2--Clause-lightgrey.svg)](https://opensource.org/license/bsd-2-clause/)


Kõik tekstides olevatad sõnavormid pole otsimisel vajalikud.
* Mõned neist on ebaolulised struktuurielemendid nagu sektsiooni numbrid.
* Mõned neist on põhjustatud töötlemisvigadest eelmistes etappides.
* Mõned sõnavormid on trükivigased või lühendid originaaltekstis, mille otsitavaks muutmiseks on vaja luua erinevaid sõnastikke.

Järgnev analüüs adresseerib kõiki neid aspekte. 

In [20]:
import re
import csv
import requests

from pandas import DataFrame 
from pandas import read_csv
from pandas import concat
from pandas import merge

from tqdm.auto import tqdm
tqdm.pandas()

## I. Puhastatud tekstide laadimine

In [2]:
sources = {}
sources['state_laws'] = read_csv('../results/cleaned_texts/state_laws.csv', header=0)
sources['government_regulations'] = read_csv('../results/cleaned_texts/government_regulations.csv', header=0)
sources['local_government_acts'] = read_csv('../results/cleaned_texts/local_government_acts.csv', header=0)
sources['government_orders'] = read_csv('../results/cleaned_texts/government_orders.csv', header=0)

## II. Indekseerimisteenuse kasutamine sõnade algvormi leidmiseks

Tekstide edasisel analüüsimisel saame kasutada veebiteennust, mis leiab igale sõnale vastava algvormi.
Kuna mõnele sõnavormile vastab mitu erinevat algvormi (`sadama` -> `sadam`, `sadama`) siis on oluline valida, kas me kasutame kõiki vorme või ühestamist, mille käigus jäetakse alles vaid konteksti sobivad algvormid. 
Erinevalt tekstide puhastamisest kasutame siin dokumentide indekseerimiseks mõeldud veebiteenust, mis jätab alles kõik algvormid, et vältida olukordi, kus algvorm leidub tekstis, aga seda ei leita ülesse.

In [3]:
NUMBER = '[0-9]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
ESTONIAN_LETTER = '[a-z|öäõüžš]'
FOREIGN_LETTER = '[ôíëаa]'

CRE_SUPERCRTIPT_ENDING = re.compile(f'(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}){SUPERSCRIPT_SYMBOLS}$') 
CRE_PSEUDO_SUPERCRTIPT_ENDING = re.compile(f'(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}){{4,}}{NUMBER}$', re.IGNORECASE) 

def index_document(text: str):
    """
    Uses web service to extract words and sub-words form document captions

    Returns a four column table with columns lemma, is_sublemma, total_weight
    where the total weight is the total count of the lemma in that role.
    The total count can be fractional as one wordform can have several lemmas 
    and the web service assigns a preference for each of them as weight.
    The sum of weights is one for the lemmas of the wordform and the higher
    weight corresponds to higher likelihood that the lemma is correct.

    If the disambiguation is set then the morphological analysis uses disambiguation
    to elliminate lemma candidates that are unlikely in the contex of the wordform.
    """
    ANALYZER_QUERY = "https://smart-search.tartunlp.ai/api/advanced_indexing/json"
    HEADERS = {"Content-Type": "application/json"}
    POST_DATA_TEMPLATE = {"params":{"tables": "indeks_lemmad"}, "sources": {"DOC_1":{"content": text}}}
    
    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    response = response.json()
    tbl = DataFrame(response['tabelid']['indeks_lemmad'], columns=['lemma', 'doc_id', 'start', 'end', 'weight', 'is_sublemma'])

    # Corrections in tokenisation. Small error in the webservice
    tbl['lemma'] = tbl['lemma'].str.strip()
    
    # Quick hack to eliminate the superscript error
    idx = tbl['lemma'].str.contains(CRE_SUPERCRTIPT_ENDING)
    tbl.loc[idx, 'lemma'] = tbl.loc[idx, 'lemma'].str[:-1]

    # Quick hack to eliminate the superscript error with CO2 exception
    idx = tbl['lemma'].str.contains(CRE_PSEUDO_SUPERCRTIPT_ENDING)
    tbl.loc[idx, 'lemma'] = tbl.loc[idx, 'lemma'].str[:-1]

    return (tbl
            .groupby(['lemma', 'is_sublemma'], as_index=False)
            .aggregate(total_weight=('weight','sum'))
            .sort_values(['lemma', 'is_sublemma']))
  

In [4]:
assert not any(index_document('1992 . aasta riigieelarveseadus')['lemma'].str.contains('\s')), 'Ootamatud tühikud'

text = 'Korruptsiooni kriminaalõigusliku reguleerimise konventsiooni ratifitseerimise seadus'
assert not any(index_document(text)['lemma'].str.contains('=', regex=False)), 'ootamatu sümbol ='

text = 'Parlamentaarsest ja valitsustevahelisest koostööst'
assert not any(index_document(text)['lemma'].str.contains('+', regex=False)), 'ootamatu sümbol +'

text = 'Seadus õunte kohta¹' 
assert not any(index_document(text)['lemma'].str.contains(CRE_SUPERCRTIPT_ENDING)), 'Subskript jääb eraldamata'

text = 'Seadus CO2 ja õunte kohta1'
assert not any(index_document(text)['lemma'].str.contains(CRE_PSEUDO_SUPERCRTIPT_ENDING)), 'Subskript jääb eraldamata'

In [5]:
display(index_document('Daam sülekoeraga ja mees jahikoeraga.').style.set_caption('Daam sülekoeraga ja mees jahikoeraga.'))
display(index_document('Punameremao. Sadama.').style.set_caption('Punameremao jalg. Sadama.'))

,lemma,is_sublemma,total_weight
0,daam,False,1.000000
1,jahikoer,False,1.000000
2,jaht,True,0.500000
3,jahtima,True,0.500000
4,koer,True,2.000000
5,mees,False,0.500000
6,mesi,False,0.500000
7,sülekoer,False,1.000000
8,süli,True,1.000000


,lemma,is_sublemma,total_weight
0,madu,True,0.500000
1,magu,True,0.500000
2,meremadu,True,0.500000
3,meremagu,True,0.500000
4,meri,True,1.000000
5,puna,True,0.500000
6,punama,True,0.500000
7,punameremadu,False,0.500000
8,punameremagu,False,0.500000
9,punameri,True,1.000000


## III. Lemmavormide täiendav puhastamine ja pealkirjade analüüs

TODO: See jutt peaks olema irrelevantne

Kuna tekstide puhastamisetapis jätsime teadlikult sisse mõned vead:

* superskripti sümbol on lisatud nimisõnale,
* tekstile järgneb sidekriipsuga number,
* valesti kirjutatud sõnad tüüpi 17-aastane,

siis nüüd oleks õige neid korrektselt käsitleda. Antud vigadest on vaja vaid parandada need, kus superskripti sümbolile järgneb number ja tekstile järgneb sidekriipsuga number. Teistel juhtudel saab indekseerija osalemmade leidmisega korrektselt hakkama.
Lihtsuse mõttes teeme asenduse peale esialgset indekseerimist. Absoluutselt korrektne lahendus oleks vastavad vead ära parandada eelmises faasis kasutades parandussõnasikku `vigane sõnavorm -> asendatav text`.

In [6]:
all_documents = concat([source for _,source in sources.items()], axis=0)

result = [None] *  len(all_documents)
for i, caption in tqdm(enumerate(all_documents['document_title']), total=len(all_documents)):
    result[i] = index_document(caption).assign(doc_id = i)

result = concat(result, axis=0).reset_index(drop=True)
display(result)

  0%|          | 0/58837 [00:00<?, ?it/s]

,lemma,is_sublemma,total_weight,doc_id
0,ajutine,False,1.0,0
1,konventsioon,False,1.0,0
2,seaduma,False,0.5,0
3,seadus,False,0.5,0
4,siss,True,0.5,0
...,...,...,...,...
773971,paadisadam,False,1.0,58836
773972,paat,True,1.0,58836
773973,piir,False,1.0,58836
773974,sadam,True,0.5,58836


### Lemmade validatsioon

In [7]:
NUMBER_SYMBOLS = '[0-9]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
ESTONIAN_LETTER = '[a-z|öäõüžš]'
FOREIGN_LETTER = '[ôíëаa]'

CRE_SUPERCRTIPT_ENDING = re.compile(f'(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}){SUPERSCRIPT_SYMBOLS}$') 
CRE_PSEUDO_SUPERCRTIPT_ENDING = re.compile(f'(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}){{4,}}{NUMBER_SYMBOLS}$', re.IGNORECASE) 

In [10]:
assert not any(result['lemma'].str.contains('\s$')), 'Ootamatud tühikud'
assert not any(result['lemma'].str.contains('^\s')), 'Ootamatud tühikud'

idx = result['lemma'].str.contains(CRE_SUPERCRTIPT_ENDING)
display(result[idx].style.set_caption('Võimalikud eraldamata supersriptid'))

idx = result['lemma'].str.contains(CRE_PSEUDO_SUPERCRTIPT_ENDING)
display(result[idx].style.set_caption('Võimalikud eraldamata pseudosupersriptid'))

,lemma,is_sublemma,total_weight,doc_id


,lemma,is_sublemma,total_weight,doc_id


### Kureeritud järjelparandused pärisnimedele


In [12]:
# Use correction map for proper names
proper_names = read_csv('inputs/proper_names.csv')
idx = result['lemma'].isin(proper_names['lemma'])
result.loc[idx, 'lemma'] = (merge(result.loc[idx, 'lemma'].reset_index(), proper_names, on='lemma', how='left')
                            .set_index('index')['korrektne_lemma'])

### Lemmade sagedustabeli loomine

In [13]:
lemma_counts = (result.groupby(['lemma', 'is_sublemma'])
                .agg(total_weight=('total_weight', 'sum'), document_count = ('doc_id', lambda x: len(set(x)))))
lemma_counts = lemma_counts.sort_values(['total_weight', 'document_count'], ascending=False).reset_index()
display(lemma_counts)

,lemma,is_sublemma,total_weight,document_count
0,kord,False,14730.000000,14452
1,põhi,True,8133.500000,7831
2,määrus,True,7173.000000,7166
3,põhimäärus,False,6949.000000,6942
4,vald,False,6732.500000,11907
...,...,...,...,...
23285,regisri,False,0.166667,1
23286,regisrine,False,0.166667,1
23287,regisris,False,0.166667,1
23288,regisrisne,False,0.166667,1


## IV. Lemmade tabeli valideerimine ja filtreerimine

Lihtsuse mõttes jätame otsitavateks lemmadeks vaid need lemmad, mis ei ole numbrid ega tähelühendid, aga võivad sisaldada erisümbolitena sidekriipse ja apostroofe.

### Lühendite ja arvude eemaldamine

Tehniliselt on lühendite nimiekirja kõige lihtsam hoida eraldi tekstifailis `abbrevations.csv` ning vajadusel seda täiendada.

In [14]:
abbrevations = read_csv('inputs/abbrevations.csv')
idx = lemma_counts['lemma'].isin(abbrevations['lühend'])
display(lemma_counts[idx].sample(10))
selected = ~idx

idx = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER_SYMBOLS}|\s)+$')
display(lemma_counts[idx & selected])
selected &= ~idx

idx = lemma_counts['lemma'].str.len() == 1
display(lemma_counts[idx & selected].sample(10))
selected &= ~ idx

,lemma,is_sublemma,total_weight,document_count
1749,IV,False,51.0,51
11200,VIII,False,2.0,2
19069,pr,False,1.0,1
15778,XXIII,False,1.0,1
4893,MHz,False,10.0,8
363,I,False,332.0,332
11644,hr,False,2.0,2
20996,õ-a,False,1.0,1
12925,sh,False,2.0,2
17265,kg,False,1.0,1


,lemma,is_sublemma,total_weight,document_count
73,2017,False,1268.0,1257
79,2020,False,1214.0,1169
105,2021,False,971.0,964
117,2019,False,871.0,858
121,2018,False,841.0,839
...,...,...,...,...
15150,931,False,1.0,1
15151,94,False,1.0,1
15153,965,False,1.0,1
15154,98,False,1.0,1


,lemma,is_sublemma,total_weight,document_count
15664,T,False,1.0,1
15716,U,False,1.0,1
12332,m,False,2.0,2
5000,A,False,9.0,9
11383,c,False,2.0,2
9259,d,False,3.0,3
10366,‑,False,3.0,3
4343,s,False,12.0,12
4892,B,False,10.0,8
21178,⁷,False,1.0,1


### Kuupäevade ja keerukamate arvudega seotud lühendite eemaldamine 


In [15]:
ESTONIAN_LETTER = '[a-z|öäõüžš]'
FOREIGN_LETTER = '[ôíëаa]'
NUMBER_SYMBOLS = '[0-9]'

# Tühikutega eraldatavad sümbolid 
PUNCTUATION_MARK = '[\\.,:;!?¿\\(\\)«»„““ˮ"‟”]'
SPECIAL_SYMBOLS = '[§/%\^]'
SUPERSCRIPT_SYMBOLS = '[⁰¹²³⁴⁵⁶⁷⁸⁹⁺⁻]'
SUBSCRIPT_SYMBOLS = '[₀₁₂₃₄₅₆₇₈₉₊₋]'

# Tühikutega asendatavad sümbolid  
WHITESPACE_SYMBOLS = '[\u200e\ufeff]'

# Teised lubatud sümbolid
DASH_SYMBOLS = '(?:‑|-|−|‒|-)'
OTHER_SYMBOLS = '[§/−a\^]'

# Type 01 . 09 . 2017
idx = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER_SYMBOLS}|,|\.|\s|%)+$')
display(lemma_counts[idx & selected])
selected &= ~idx

# Type 10а
idx = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER_SYMBOLS})+(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})$', case=False) 
display(lemma_counts[idx & selected])
selected &= ~idx

# Type 813-k
idx = lemma_counts['lemma'].str.contains(f'^(?:{NUMBER_SYMBOLS})+{DASH_SYMBOLS}(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})$', case=False) 
display(lemma_counts[idx & selected])
selected &= ~idx

,lemma,is_sublemma,total_weight,document_count
6983,01 . 09 . 2017,False,5.0,5
6989,31 . 12 . 2018,False,5.0,5
7839,"1 , 4",False,4.0,4
7842,22 . 02 . 2018,False,4.0,4
7843,"3 , 5",False,4.0,4
...,...,...,...,...
15045,31 . 12 . 2017,False,1.0,1
15065,"4 , 5",False,1.0,1
15097,"56 , 8",False,1.0,1
15122,"7 , 9",False,1.0,1


,lemma,is_sublemma,total_weight,document_count
14946,10а,False,1.0,1
14980,18a,False,1.0,1
14992,1a,False,1.0,1
15046,313a,False,1.0,1
15064,3M,False,1.0,1
15069,43A,False,1.0,1
15084,4a,False,1.0,1
15137,80s,False,1.0,1
15156,9c,False,1.0,1


,lemma,is_sublemma,total_weight,document_count
15112,620-k,False,1.0,1
15140,813-k,False,1.0,1
15152,94-k,False,1.0,1


Allesjäänud numbreid sisaldavad sõnu ei saa algoritmiliselt paremini töödelda. 
Seetõttu moodustame eraldi lubatud sõnade loendi ja jätame alles vaid need numbreid sisaldavad lemmad, mis on kureeritud failis `allowed_names.csv`.

In [16]:
allowed_names = read_csv('inputs/allowed_names.csv')

idx = lemma_counts['lemma'].str.contains('[0-9]')
display(lemma_counts[idx & selected].sample(10))
idx &= ~ lemma_counts['lemma'].isin(allowed_names['lemma'])
lemma_counts.loc[idx & selected, 'lemma'].to_csv('../results/candidate_lists/allowed_names.csv', index=False)

selected &= ~idx

,lemma,is_sublemma,total_weight,document_count
15671,TKNE-7,False,1.0,1
15083,4S3,False,1.0,1
14939,1000ruut,True,1.0,1
15211,C1-alamkategooria,False,1.0,1
11001,24meetrine,False,2.0,2
15210,C1-alam,True,1.0,1
15218,CO2,False,1.0,1
15107,5kroonine,False,1.0,1
15432,L7,False,1.0,1
15196,B1-alam,True,1.0,1


### Allesjäänud lühendite sõnaraamatu koostamine

Lemmade hulka alles jäänud lühenditest on kasulik luua loend faili `abbrevation_dictionary.csv`. See võimaldab meil hiljem tekitada sõnastiku, mille abil pikk otsisõna laiendatakse ka vastavaks lühendiks näiteks Euroopa Liit → EL ja aksiaselts → AS. 

In [17]:
idx = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER.upper()}|{FOREIGN_LETTER.upper()}|{NUMBER_SYMBOLS})+$')
display(lemma_counts.loc[idx & selected].sample(10))

(lemma_counts.loc[idx & selected, ['lemma']]
 .rename(columns={'lemma': 'lühend'})
 .assign(täisnimi='')
 .sort_values(['lühend'])
 .to_csv('../results/candidate_lists/abbrevation_dictionary.csv', index=False))

,lemma,is_sublemma,total_weight,document_count
15627,SE,False,1.0,1
11068,EMKO,False,2.0,2
15213,CE,False,1.0,1
15261,EST,False,1.0,1
3876,NATO,False,15.0,12
15253,EASA,False,1.0,1
15665,TALO,False,1.0,1
15717,UMTS,False,1.0,1
15474,MAK,False,1.0,1
6992,DNA,False,5.0,5


### Pärisnimede ja initsiaalide korrektne töötlemine

Pärisnimede korral ei toimi lemmatiseerimine alati korrektselt ja seega tuleb need käsitsi läbi vaadata. Absoluutselt korrektne viis pärisnimede tuvastamiseks on nimeolemituvastus. Näiteks [TartuNLP nimeolemituvastus](https://ner.tartunlp.ai) või [EstNLTK nimeolemituvastus](https://github.com/estnltk/estnltk/blob/main/tutorials/nlp_pipeline/D_information_extraction/02_named_entities.ipynb). Antud kontekstis on kõige lihtsam otsida vaid initsiaalidega pärisnimesid, sest nende analüüs on kõige suurem probleem. Nagu eelnevalt teeme vastava manuaalselt kureeritud faili `proper_names.csv`.

In [21]:
idx = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER})\s\.', case=False)
idx &= ~lemma_counts['is_sublemma']
display(lemma_counts[idx & selected])

(lemma_counts
 .loc[idx & selected & ~lemma_counts['lemma'].isin(proper_names['korrektne_lemma']), ['lemma']]
 .assign(korrektne_lemma=lambda df: df['lemma'].str.title())
 .to_csv('../results/candidate_lists/proper_names.csv', index=False, quoting=csv.QUOTE_ALL))

,lemma,is_sublemma,total_weight,document_count
3069,F . J . Wiedemann,False,21.0,21
5842,J . V . Veski,False,7.0,7
7851,C . R . Jakobson,False,4.0,4
7874,M . Lüdig,False,4.0,4
9059,A . H . Tammsaare,False,3.0,3
11146,O . Luts,False,2.0,2
11170,S . Miloševic,False,2.0,2
15157,A . Hahn,False,1.0,1
15244,E . Vilde,False,1.0,1
15305,G . Ots,False,1.0,1


Eemaldame initsiaalid otsitavate alamsõnade hulgast. Seda saaks paremini teha, aga see on piisavlt täpne muster.

In [22]:
idx = lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|\s|\.)+$', case = False)
idx &= lemma_counts['lemma'].str.contains('\s')
idx &= lemma_counts['is_sublemma']
display(lemma_counts[idx])
selected &= ~ idx

,lemma,is_sublemma,total_weight,document_count
3068,F . J .,True,21.0,21
5841,J . V .,True,7.0,7
7850,C . R .,True,4.0,4
9058,A . H .,True,3.0,3
9060,A . H . tamm,True,3.0,3
9084,Fr . R .,True,3.0,3
11145,O .,True,2.0,2
15243,E .,True,1.0,1
15304,G .,True,1.0,1


In [23]:
lemma_counts[lemma_counts['lemma'].str.contains('\s') &selected]

,lemma,is_sublemma,total_weight,document_count
3069,F . J . Wiedemann,False,21.0,21
5842,J . V . Veski,False,7.0,7
7851,C . R . Jakobson,False,4.0,4
7874,M . Lüdig,False,4.0,4
9059,A . H . Tammsaare,False,3.0,3
9085,Fr . R . Kreutzwald,False,3.0,3
11146,O . Luts,False,2.0,2
11170,S . Miloševic,False,2.0,2
15157,A . Hahn,False,1.0,1
15244,E . Vilde,False,1.0,1


**Kommentaar:** Tulemus näitab, et esialgne sõnestus sisaldab vigu veebiaadresside osas. 
Meie korrektsioonid enne sõnestamist löövad veebiaadressi kujule xyz . abc . com, mis tekitab väga palju lühikesi sõnu.
Seda on võimalik parandada täpsema eeltöötlusega. Samamoodi on lahti löödud lühendid nagu P.L.C. ja U.K.

### Sidekriipsuga sõnade analüüs

Liitsõnad sisaldavad ainult sidekriipsu mitte teisi sidekriipsulaadseid sümboleid.  
Vaatlus näitab, et peale nimede lemmatiseerimis- ja suurtähevigade muid kahtlasi olukordi pole.  

In [24]:
idx = lemma_counts['lemma'].str.contains(DASH_SYMBOLS)
idx &= ~lemma_counts['lemma'].str.contains('-')
display(lemma_counts[selected & idx])

,lemma,is_sublemma,total_weight,document_count


In [25]:
idx = lemma_counts['lemma'].str.contains(DASH_SYMBOLS)
idx &= lemma_counts['lemma'].map(lambda x: x[0].isupper())
display(lemma_counts[selected & idx])

,lemma,is_sublemma,total_weight,document_count
346,Kohtla-Järve,False,353.000000,349
571,Narva-Jõesuu,False,210.000000,210
572,Narva-Jõgi,True,210.000000,210
638,Väike-Maarja,False,190.000000,189
765,Lääne-Nigula,False,154.000000,152
...,...,...,...,...
22755,Mullutu-Lood,False,0.333333,1
22756,Mullutu-Loode,False,0.333333,1
22805,Vääna-Vit,False,0.333333,1
22806,Vääna-Viti,False,0.333333,1


In [26]:
idx = lemma_counts['lemma'].str.contains(DASH_SYMBOLS)
idx &= ~lemma_counts['lemma'].map(lambda x: x[0].isupper())
display(lemma_counts[selected & idx].sample(10))

,lemma,is_sublemma,total_weight,document_count
18915,pedagoog-metoodik,False,1.0,1
10687,masin-kuivati,True,2.5,3
12022,kool-Instituut,True,2.0,2
15974,arengu-eelarvekomisjon,False,1.0,1
11232,aed-põhikool,True,2.0,2
10841,hoiu-laen,True,2.0,4
16068,audio-videotehnika,False,1.0,1
16878,juhendajate-õpetaja,False,1.0,1
7696,raudtee-esi,True,4.5,7
17791,kütte-piirkond,True,1.0,1


### Ootamatuid sümboleid sisaldavate sõnade analüüs

In [27]:
idx = ~lemma_counts['lemma'].str.contains(f'^(?:{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{DASH_SYMBOLS}|{NUMBER_SYMBOLS})+$', case=False)
idx &= ~lemma_counts['lemma'].isin(proper_names['korrektne_lemma']) 
display(lemma_counts[idx & selected])

,lemma,is_sublemma,total_weight,document_count
16227,eesti . ee,False,1.0,1


### Trükivigadele vastavad lemmad

Nii imelik kui see ka pole, siis Riigi Teataja pealkirjade hulgas esineb ka trükivigu. Neid on kõige lihtsam tuvastada selle järgi, et need esinevad harva ja need pole eestikeelsete sõnade nimistus. Nagu eelnevalt teeme vastava manuaalselt kureeritud faili `misspellings.csv`.

In [30]:
rare_words = lemma_counts[selected & (lemma_counts['total_weight'] < 3) & (lemma_counts['document_count'] <= 3)]

In [31]:
def is_known_word(text: str):
    """
    Uses web service to validate if the word is inside Vabamorf dictionary
    """

    ANALYZER_QUERY = "https://smart-search.tartunlp.ai/api/analyser/process"
    HEADERS = {"Content-Type": "application/json; charset=utf-8"}
    POST_DATA_TEMPLATE = {'params': {}, 'content': text} 

    response = requests.post(ANALYZER_QUERY, json=POST_DATA_TEMPLATE, headers=HEADERS)
    assert response.ok, "Webservice failed"
    response = response.json()
    return  'mrf' in response['annotations']['tokens'][0]['features']
 
print(is_known_word('kala'))
print(is_known_word('regisri'))

True
False


In [33]:
idx = rare_words['lemma'].progress_map(is_known_word)
name_candidates = rare_words['lemma'].str.contains(f'^{ESTONIAN_LETTER.upper()}|{FOREIGN_LETTER.upper()}')
display(rare_words[~idx & ~name_candidates])

(rare_words
 .loc[~idx & ~name_candidates, ['lemma']]
 .assign(korrektne_lemma=lambda df: df['lemma'])
 .sort_values(by=['lemma'])
 .to_csv('../results/candidate_lists/misspellings.csv', index=False))

  0%|          | 0/11875 [00:00<?, ?it/s]

,lemma,is_sublemma,total_weight,document_count
10965,saastum,True,2.000000,3
10966,saastumi,True,2.000000,3
10967,saastumis,True,2.000000,3
10969,satelliitjälgimi,True,2.000000,3
10971,territoriaa,False,2.000000,3
...,...,...,...,...
23285,regisri,False,0.166667,1
23286,regisrine,False,0.166667,1
23287,regisris,False,0.166667,1
23288,regisrisne,False,0.166667,1


## V. Indeksisse minevate lemmade eksportimine

In [34]:
VALID_SYMBOL = f'{ESTONIAN_LETTER}|{FOREIGN_LETTER}|{DASH_SYMBOLS}|{NUMBER_SYMBOLS}| |\.'
idx = lemma_counts['lemma'].str.contains(f'^(?:{VALID_SYMBOL})+$', case=False)
assert not any(selected & ~idx), "Ootamatus sümbolid valitud sõnade hulgas"

outcome = lemma_counts[idx & selected]
print(f'Täislemmade arv:      {sum(~outcome["is_sublemma"])}')
print(f'Alamsõnavoormide arv: {sum(outcome["is_sublemma"])}')


Täislemmade arv:      15565
Alamsõnavoormide arv: 7251


In [35]:
outcome = (outcome
           .rename(columns={'total_weight': 'occurence_count'})
           .assign(occurence_count=lambda df: df['occurence_count'].round(decimals=1))
           .sort_values(['occurence_count', 'document_count'], ascending=False)
           [['is_sublemma', 'lemma', 'occurence_count', 'document_count']])

outcome.to_csv('../results/word_indices/lemma_index.csv', header=True, index=False)
display(outcome)

,is_sublemma,lemma,occurence_count,document_count
0,False,kord,14730.0,14452
1,True,põhi,8133.5,7831
2,True,määrus,7173.0,7166
3,False,põhimäärus,6949.0,6942
4,False,vald,6732.5,11907
...,...,...,...,...
23285,False,regisri,0.2,1
23286,False,regisrine,0.2,1
23287,False,regisris,0.2,1
23288,False,regisrisne,0.2,1
